In [5]:
using Distributions, Plots

INFO: Recompiling stale cache file /home/keorn/.julia/lib/v0.6/Plots.ji for module Plots.


In [6]:
candidate_n = 50 # Number of validator candidates
validator_n = 10 # Number of validator slots
nominator_n = 100; # Number of nominator candidates

In [7]:
# Nominator preferences, determined by their stake, preferred return and validators
function nomination_preference(ordered_candidates::Vector)
    l = length(ordered_candidates)
    rand(1:l, rand(1:l))
end

nomination_preference (generic function with 1 method)

In [120]:
candidate_stake(candidate_n::Int) = zeros(candidate_n) # Validators represented only by their stake, always ordered
#candidate_stake(candidate_n::Int) = sort(rand(Geometric(0.01), candidate_n)) # Validators represented only by their stake, always ordered
nominator_stake(nominator_n::Int) = sort(rand(Geometric(0.01), nominator_n)) # Nominator stakes
nominator_interest(nominator_n::Int) = rand(Beta(5, 20), nominator_n) # Desired nominator interests

nominator_interest (generic function with 1 method)

In [136]:
# Total stake
total_stake(validators::Vector, nominations) = validators + sum(nominations, 2)[:]
inflation(stakes::Vector, rates::Vector) = (rates' * stakes)[1]
inflation(validators::Vector, nominations, rates::Vector) = inflation(total_stake(validators, nominations), rates)
# Does not account for validator stake
inflation(interests::Vector, nominations::Array{Float64, 2}) = (sum(nominations, 1) * interests)[1]
# Threshold
threshold(validator_n::Int, candidates::Vector, nominations) = minimum(sort(sum(nominations, 2)[:]+candidates)[end-validator_n+1:end])

threshold (generic function with 1 method)

In [137]:
struct World
    validator_n::Int
    candidate_n::Int
    candidates::Vector{Int}
    nominator_n::Int
    nominators::Vector{Int}
    interests::Vector{Float64}
    preferences::Vector{Vector{Int}}
end
function World(validator_n::Int, candidate_n::Int, nominator_n::Int)
    candidates = candidate_stake(candidate_n)
    World(
        validator_n,
        candidate_n,
        candidates,
        nominator_n,
        nominator_stake(nominator_n),
        nominator_interest(nominator_n),
        [nomination_preference(candidates) for _ in 1:nominator_n]
    )
end

World

In [138]:
function sample_worlds(validator_n::Int, candidate_n::Int, nominator_n::Int, sample_n::Int)::Vector{World}
    [World(validator_n, candidate_n, nominator_n) for _ in 1:sample_n]
end
function opt_rates(strat::Function, w::World)
    rates, nominations = strat(w)
    available_stake = sum(w.nominators)+sum(w.candidates)
    [inflation(w.candidates, nominations, rates)/available_stake,
        threshold(validator_n, w.candidates, nominations)/available_stake]
end
function opt_rates(strat::Function, ws::Vector{World})
    res = hcat([opt_rates(strat, w) for w in ws]...)'
    (res[:, 1], res[:, 2])
end

opt_rates (generic function with 2 methods)

In [139]:
function opt(strat::Function, w::World)
    _, nominations = strat(w)
    available_stake = sum(w.nominators)+sum(w.candidates)
    [inflation(w.interests, nominations)/available_stake,
        threshold(validator_n, w.candidates, nominations)/available_stake]
end
function opt(strat::Function, ws::Vector{World})
    res = hcat([opt(strat, w) for w in ws]...)'
    (res[:, 1], res[:, 2])
end

opt (generic function with 2 methods)

In [140]:
ws = sample_worlds(validator_n, candidate_n, nominator_n, 1000);

In [141]:
function brute_strat(w::World)
    rates = rand(Beta(5, 20), w.candidate_n)
    nominations = zeros(w.candidate_n, w.nominator_n)
    subset = rand(1:w.candidate_n, w.validator_n)
    for n in 1:w.nominator_n
        preference = filter(p -> p in subset, w.preferences[n])
        c = length(preference)
        for p in preference
            if rates[p] >= w.interests[n]
                nominations[p, n] += w.nominators[n]/c
            end
        end
    end
    (rates, nominations)
end

brute_strat (generic function with 1 method)

In [142]:
function adarates_strat(w::World)
    rates = zeros(w.candidate_n)
    nominations = zeros(w.candidate_n, w.nominator_n)
    subset = rand(1:w.candidate_n, w.validator_n)
    for n in 1:w.nominator_n
        preference = filter(p -> p in subset, w.preferences[n])
        c = length(preference)
        for p in preference
            if w.interests[n] > rates[p]
                rates[p] = w.interests[n]
            end
            nominations[p, n] += w.nominators[n]/c
        end
    end
    (rates, nominations)
end

adarates_strat (generic function with 1 method)

In [143]:
function th_strat(w::World)
    t = (sum(w.nominators)+sum(w.candidates[1:w.validator_n]))/w.validator_n
    rates = zeros(candidate_n)
    nominations = zeros(w.candidate_n, w.nominator_n)
    for n in 1:nominator_n
        remaining_nomination = w.nominators[n]
        for p in w.preferences[n]
            existing_stake = sum(nominations[p, :]) + w.candidates[p]
            diff = t - existing_stake
            if diff > 0
                if w.interests[n] > rates[p]
                    rates[p] = w.interests[n]
                end
                if diff >= remaining_nomination
                    nominations[p, n] += remaining_nomination
                    break
                else
                    nominations[p, n] += diff
                    remaining_nomination -= diff
                end
            end
        end
    end
    (rates, nominations)
end

th_strat (generic function with 1 method)

In [144]:
function greedy_strat(w::World)
    thr = round(Int, (sum(w.nominators)+sum(w.candidates[1:w.validator_n]))/w.validator_n)
    target_rate = 0.275 #rand(Beta(5, 20))
    available = Dict{Int, Set{Int}}()
    for n in 1:w.nominator_n
        if w.interests[n] <= target_rate
            for p in w.preferences[n]
                if haskey(available, p)
                    push!(available[p], n)
                else
                    available[p] = Set(n)
                end
            end
        end
    end
    nominations = zeros(w.candidate_n, w.nominator_n)
    for v in 1:w.validator_n
        if isempty(available) break end
        max_stake = 0
        max_c = 0
        for (c, ns) in available
            s = sum(w.nominators[n] for n in ns)
            if s > max_stake
                max_stake = s
                max_c = c
            end
        end
        if max_c == 0 break end
        assigned_ns = Set{Int}()
        remaining_stake = thr - w.candidates[max_c]
        for n in available[max_c]
            available_amount = w.nominators[n]
            diff = available_amount - remaining_stake
            if diff <= 0
                push!(assigned_ns, n)
                remaining_stake -= available_amount
                nominations[max_c, n] = available_amount
            else
                nominations[max_c, n] += diff
                w.nominators[n] -= diff
                break
            end
        end
        for (c, ns) in available
            setdiff!(ns, assigned_ns)
            if isempty(ns) delete!(available, c) end
        end
        delete!(available, max_c)
    end
    (fill(target_rate, candidate_n), nominations)
end

greedy_strat (generic function with 1 method)

In [145]:
function greedyada_strat(w::World)
    thr = round(Int, (sum(w.nominators)+sum(w.candidates[1:w.validator_n]))/w.validator_n)
    target_rate = 0.3 #rand(Beta(5, 20))
    available = Dict{Int, Set{Int}}()
    rates = zeros(candidate_n)
    for n in 1:w.nominator_n
        if w.interests[n] <= target_rate
            for p in w.preferences[n]
                if haskey(available, p)
                    push!(available[p], n)
                else
                    available[p] = Set(n)
                end
            end
        end
    end
    nominations = zeros(w.candidate_n, w.nominator_n)
    for v in 1:w.validator_n
        if isempty(available) break end
        max_stake = 0
        max_c = 0
        for (c, ns) in available
            s = sum(w.nominators[n] for n in ns)
            if s > max_stake
                max_stake = s
                max_c = c
            end
        end
        if max_c == 0 break end
        assigned_ns = Set{Int}()
        remaining_stake = thr - w.candidates[max_c]
        for n in available[max_c]
            if w.interests[n] > rates[max_c]
                rates[max_c] = w.interests[n]
            end
            available_amount = w.nominators[n]
            diff = available_amount - remaining_stake
            if diff <= 0
                push!(assigned_ns, n)
                remaining_stake -= available_amount
                nominations[max_c, n] = available_amount
            else
                nominations[max_c, n] += diff
                w.nominators[n] -= diff
                break
            end
        end
        for (c, ns) in available
            setdiff!(ns, assigned_ns)
            if isempty(ns) delete!(available, c) end
        end
        delete!(available, max_c)
    end
    (rates, nominations)
end

greedyada_strat (generic function with 1 method)

In [131]:
th = opt_rates(th_strat, ws)
adarates = opt_rates(adarates_strat, ws);
greedyada = opt_rates(greedyada_strat, ws);
greedy = opt_rates(greedy_strat, ws);

In [146]:
th = opt(th_strat, ws)
adarates = opt(adarates_strat, ws);
greedyada = opt(greedyada_strat, ws);
greedy = opt(greedy_strat, ws);

In [147]:
mean(th[1]), std(th[1]), mean(th[2]), std(th[2])

(0.19944699961553256, 0.011192887360683032, 0.03535630700148179, 0.0032517079292774713)

In [148]:
mean(adarates[1]), std(adarates[1]), mean(adarates[2]), std(adarates[2])

(0.18178688715716743, 0.013669434241529605, 0.022607358381599087, 0.029461759167243515)

In [151]:
# X is inflation, Y is threshold
scatter(th, label = "th", xaxis = "Inflation", yaxis = "Threshold")
#scatter!(adarates, label = "adarates")
#scatter!(opt_rates(brute_strat, ws), label = "brute")
scatter!(greedy, label = "greedy")
scatter!(greedyada, label = "greedyada")